In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
root = './data'
if not os.path.exists(root):
    os.mkdir(root)

In [ ]:
train_set = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
test_set = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
import random

tensor_to_PIL = transforms.ToPILImage()

rand_idx_lst = random.sample(range(50001), 10)
fig, axes = plt.subplots(1, 10, figsize=(20, 5))

for idx in range(len(axes)):
    axes[idx].imshow(tensor_to_PIL(train_set[rand_idx_lst[idx]][0]))

plt.show()
plt.close()

In [ ]:
batch_size = 100
total_epoch = 10
learning_rate = 0.01
use_cuda = torch.cuda.is_available()

In [ ]:
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

In [ ]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def name(self):
        return "MLP"

In [ ]:
model = MLPNet()
if use_cuda:
    model = model.cuda()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(total_epoch):
    # trainning
    total_loss = 0
    total_batch = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()

        out = model(x)
        loss = criterion(out,target)
        total_loss += loss.item()
        total_batch += 1
        loss.backward()
        optimizer.step()

    # testing
    total_loss = 0
    total_batch = 0
    correct_cnt = 0
    total_cnt = 0

    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
            x, target = x.cuda(), target.cuda()

        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum().item()

        total_loss += loss.item()
        total_batch += 1
        if (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, test loss: {:.6f}, acc: {:.3f}'
                   .format(epoch, total_loss / total_batch, correct_cnt * 1.0 / total_cnt))

In [ ]:
rand_idx_lst = random.sample(range(1000), 10)
fig, axes = plt.subplots(1, 10, figsize=(20, 5))

pred_labels = []
for idx in range(len(axes)):
    x = test_set[rand_idx_lst[idx]][0]
    axes[idx].imshow(tensor_to_PIL(x))
    if use_cuda:
        x = x.cuda()
    out = model(x)
    _, pred_label = torch.max(out.data, 1)
    pred_labels.append(pred_label.item())

plt.show()
plt.close()
print(pred_labels)